# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [ ]:
DROP TABLE IF EXISTS view_grupo_ponte;

CREATE VIEW view_grupo_ponte AS
SELECT 
  p.code,
  p.popularity,
  i.total_consumption,
  i.average_consumption,
  i.average_consumption_bw,
  r.recipes
FROM 
  (SELECT FCID_Code AS code, COUNT(DISTINCT SeqN) AS popularity FROM Intake GROUP BY FCID_Code) p
JOIN
  (SELECT FCID_Code, SUM(Intake) AS total_consumption, AVG(Intake) AS average_consumption, AVG(intake_bw) AS average_consumption_bw FROM Intake GROUP BY FCID_Code) i ON p.code = i.FCID_Code
JOIN
  (SELECT FCID_Code, COUNT(*) AS recipes FROM Recipes GROUP BY FCID_Code) r ON p.code = r.FCID_Code;

SELECT * FROM view_grupo_ponte;


## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

In [ ]:
-- Para explorar a relação entre as métricas, utilizaremos a função CORR do SQL, que calcula um valor que chamamos de coeficiente de correlação. 
-- O valor do coeficiente varia de -1 a 1. Valores próximos de 1 significa correlação positiva, enquanto valores perto de -1 indicam correlação negativa. Valores próximos de 0 indicam a não-correlação.

SELECT CORR(popularity, total_consumption) AS Correlation FROM view_grupo_ponte;
SELECT CORR(popularity, average_consumption) AS Correlation FROM view_grupo_ponte;

## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

In [ ]:
-- Sim, podemos. A query abaixo por exemplo, pega quais os alimentos mais consumidos por cada consumidor.
-- Isso por si só já traz o perfil de cada consumidor, pois sabemos qual seu alimento "favorito".


WITH consumo_max AS (
  SELECT SeqN, FCID_Code, MAX(Intake) AS max_consumido
  FROM Intake
  GROUP BY SeqN, FCID_Code
)


-- E na query abaixo, por exemplo, agrupamos os consumidores que tem como alimento mais consumido, 
-- um alimento com Cooked_status=1.
SELECT COUNT(*) AS soma_consumidores
FROM (
  SELECT i.SeqN, i.FCID_Code, mc.Max_Intake
  FROM Intake i
  JOIN consumo_max mc ON i.SeqN = mc.SeqN AND i.FCID_Code = mc.FCID_Code
  WHERE i.Cooked_Status = 1
) AS Alimentos_Cooked_Status_1;


## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

In [ ]:
-- Podemos usar a mesma do exercício 3
SELECT SeqN, FCID_Code, MAX(Intake) AS max_consumido
FROM Intake
GROUP BY SeqN, FCID_Code